## 2D Grid

We'll cover the following topics in this notebook:
- Modules and signatures
- Grid as list of lists
- OCaml arrays
- Grid as OCaml array


### Modules

When our code base grows bigger, we need a way of organizing data structures and their corresponding operations. Module system is the solution. We start with writing a library of queue, implemented with lists.

In [ ]:
type 'a queue = 'a list * 'a list
exception Empty

let empty = [], []

let null = function
  | [], [] -> true
  | _ -> false

let norm = function
  | inq, [] -> [], List.rev inq
  | q -> q

let enq a (inq, outq) = norm (a :: inq, outq)

let deq = function
  | inq, hd :: outq -> norm (inq, outq)
  | _ -> raise Empty

let top = function
  | _, hd :: _ -> hd
  | _ -> raise Empty

let size (inq, outq) = List.length inq + List.length outq

In [ ]:
let q = empty
let q = enq 3 q
let q = enq 4 q
let x = hd (snd q)

 There are several problems:
- The types of the operations are still referring to `'a list`, not `'a queue`.
- Future definitions cannot use simple names like `empty` or `length`.
- A queue, ideally, should only be modified via `enq` and `deq`, but the user can manipulate the queue arbitrarily like manipulating a list.

We can solve the first problem by adding type annotations, and the second problem by using `long_and_complicated_function_names`. For the last problem, we can only ask the user politely to not manipulate the queue. 

The key problem here is that we need a suitable method of *abstraction*, a module that packs the type definitions and functions together and hides the implementation details.

In [ ]:
module type Queue = 
	sig
	  type 'a queue
	  exception Empty
	  val empty : 'a queue
	  val null : 'a queue -> bool
	  val enq : 'a -> 'a queue -> 'a queue
	  val deq : 'a queue -> 'a queue
	  val top : 'a queue -> 'a
	  val size : 'a queue -> int
	end

Signatures are interfaces for structures. A signature specifies which components of a structure are accessible from the outside, and with which type.

In [ ]:
module Q : Queue =
	struct
		type 'a queue = 'a list * 'a list
	  exception Empty

	  let empty = [], []

	  let null = function
	    | [], [] -> true
	    | _ -> false

	  let norm = function
	    | inq, [] -> [], List.rev inq
	    | q -> q

	  let enq a (inq, outq) = norm (a :: inq, outq)

	  let deq = function
	    | inq, hd :: outq -> norm (inq, outq)
	    | _ -> raise Empty

	  let top = function
	    | _, hd :: _ -> hd
	    | _ -> raise Empty

	  let size (inq, outq) = List.length inq + List.length outq
	end

Modules use `struct...end` syntax to group related definations together. By restricting the signature of `Q` to `Queue`, we forbid functions from accessing the underlying strudture of our queues, i.e. lists.

In [ ]:
let q = Q.empty
let q = Q.enq 3 q
let q = Q.enq 4 q
let x = hd (snd q) (* This won't work *)

### 2D Grid

We now look at a signature of a 2-dimensional grid, which we use to model a map of trees or open grounds. We use a pair of integers (the coordinate) to represent a position on the map.


In [ ]:
type position = int * int
type item = Tree | Ground


We now define the signature of the grid. It should contain all data we need to represent the grid (width, height, what's on a position) and operations for the map (init, update). We might as well add a display function for printing the grid out.

In [ ]:
let show_item = function
	| Tree -> "X"
	| Ground -> "O"

module type Grid = 
	sig
		type grid

		(* Basic information *)
		val width : grid -> int
		val height : grid -> int
		val item : grid -> position -> item

		(* Creation and update *)
		val init : int -> int -> (int -> int -> item) -> grid
		val update : grid -> position -> item -> grid
	
		(* Display *)
		val show : grid -> unit
	end

Using what we've learned so far, we can implement the grid using a list of lists.

In [ ]:
module ListGrid : Grid = 
	struct
		type grid = item list list * int * int

		let width (gr, w, h) = w
		let height (gr, w, h) = h
		let item (gr, w, h) (x, y) = 
			List.nth (List.nth gr x) y

		let init w h f = 
			let grid = List.init w (fun x -> List.init h (fun y -> f x y)) in
				(grid, w, h)

		let update (gr, w, h) (x, y) v = 
			let rec update_list x v i = function
				| [] -> []
				| hd :: tl  -> if i = x then v :: tl else hd :: update_list x v (i + 1) tl
			in
			let new_row = update_list x v 0 (List.nth gr y) in
			let new_grid = update_list y new_row 0 gr in
					(new_grid, w, h)

		let to_string gr = 
			let fold_row str row = 
				List.fold_left (fun acc item -> acc ^ (show_item item) ^ " ") str row
			in
				List.fold_left (fun acc row -> fold_row acc row ^ "\r\n" ) "" gr
		
		let show (gr, w, h) = print_endline (to_string gr)

	end

In [ ]:
let g1 = ListGrid.init 3 8 (fun x y -> Ground)
let () = ListGrid.show g1

### OCaml Arrays

Apart from using functional arrays, OCaml provides actual mutable arrays like those in C or C++. Here are common ways to create, access, and modify an array, and you can find more in the [documentation](https://ocaml.org/manual/5.3/api/Array.html).

In [ ]:
(* Create array *)
let arr1 = [|1; 2; 3; 4; 5|]
let arr2 = Array.init 10 (fun x -> x + 3)
let arr3 = Array.of_list ['a'; 'b'; 'c']

(* Array access *)
let x = arr1.(3)
let y = Array.get arr2 5
let z = Array.get arr3 10

(* Array update *)
(* Note the type of these! *)
let () = arr1.(3) <- 10
let () = Array.set arr2 5 100
let () = Array.set arr3 10 10

We can now use the imperative features of OCaml to implement a grid with arrays of arrays.

In [ ]:
module ArrayGrid : Grid = 
	struct 
		type grid = item array array * int * int

		let width (gr, w, h) = w
		let height (gr, w, h) = h
		let item (gr, w, h) (x, y) = gr.(x).(y)

		let init w h f = 
			let arr = Array.init w (fun x -> Array.init h (fun y -> f x y)) in
				(arr, w, h)

		let update (gr, w, h) (x, y) item = 
			gr.(x).(y) <- item; 
			(gr, w, h)

		let show (gr, w, h) = 
			let str = ref ("") in

			for x = 0 to (w - 1) do
				for y = 0 to (h - 1) do
					str := !str ^ (show_item gr.(x).(y)) ^ " "
				done;
				str := !str ^ "\n"
			done;

			print_endline !str

	end

In [ ]:

let g2 = ArrayGrid.init 4 5 (fun x y -> (if x + y = 7 then Tree else Ground))